In [2]:
import pandas as pd
import math
import random


In [3]:
house_votes = pd.read_csv("house-votes-84.data.csv",na_values="?")
# print(house_votes.iloc[107].to_string())
df_size = len(house_votes)
classes_names = house_votes.columns.to_list()[1:]
# house_votes = pd.DataFrame(house_votes.drop([107,248]))
# house_votes.iloc[107]
house_votes.iloc[107]
house_votes[house_votes.isna().any(axis=1)]

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,democrat,NaN,NaN,NaN,n,n,n,y,y,y,y,n,n,y,n,y,y
429,democrat,y,n,y,n,NaN,n,y,y,y,y,n,y,n,NaN,y,y
432,republican,n,NaN,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,NaN,NaN,NaN,NaN,n,y,y,y,n,y


In [4]:
def train_NB(df: pd.DataFrame) :
    prior = df.groupby("Class Name").size().div(len(df))
    conditional_probability = {}
    for c in classes_names:
        conditional_probability[c] = df.groupby(["Class Name",c]).size().div(len(df)).div(prior)
    return prior,conditional_probability

In [5]:
### DATA PRE-PROCESSING
prior, cond_prob = train_NB(house_votes)
print(cond_prob["water-project-cost-sharing"]["republican"])
filled_df = house_votes.copy()
for i in filled_df.columns:
    filled_df[i].fillna(filled_df[i].mode()[0], inplace=True)
# filled_df.iloc[107]


water-project-cost-sharing
n    0.434524
y    0.446429
dtype: float64


In [6]:

def apply_NBC(target_attribute,prior,condtitional_probability,query):
    score = {}
    for target_class in target_attribute:
        score[target_class] = math.log(prior[target_class])
        for attribute in classes_names:
            if pd.isna(query[attribute]) :
                continue
            cond_prob_value = condtitional_probability[attribute][target_class][query[attribute]]
            score[target_class] += math.log(cond_prob_value)
    return max(score,key = lambda k: score[k])

In [7]:
prior, cond_prob = train_NB(house_votes)
hits = 0

target_attribute = house_votes["Class Name"].unique()
for i in range(df_size):
    guess = apply_NBC(target_attribute,prior,cond_prob,house_votes.iloc[i])
    if guess == house_votes.iloc[i]["Class Name"]:
        hits += 1
print(hits/df_size)
# print(apply_NBC(target_attribute,prior,cond_prob,house_votes.iloc[107][classes_names]),house_votes.iloc[107]["Class Name"])
# print(apply_NBC(target_attribute,prior,cond_prob,house_votes.iloc[1]),house_votes.iloc[1]["Class Name"])

0.903448275862069


In [8]:
def get_acc(prior,cond_prob,test_set):
    target_attribute = test_set["Class Name"].unique()
    hits = 0
    count = len(test_set)
    for i in range(len(test_set)):
        guess = apply_NBC(target_attribute,prior,cond_prob,test_set.iloc[i])
        if guess == test_set.iloc[i]["Class Name"]:
            hits+=1
    # print(hits/count)
    return (hits/count)

In [14]:
def test_program(df:pd.DataFrame):

    sets = [[] for _ in range(10)]
    df_size = len(df)
    for i in range(df_size):
        random.choice(sets).append(i)
    results = 0
    for set in sets:
        train_set = df.drop(set).reindex()
        test_set = df.iloc[set].reindex()
        prior, condprob = train_NB(train_set)

        results +=get_acc(prior,condprob,test_set)
    results /=10
    # print("total",results)
    return results

In [16]:
virgin_data_mean_result = 0
filled_data_mean_result = 0
for i in range(5):
    # virgin_data_mean_result += test_program(house_votes)
    filled_data_mean_result += test_program(filled_df)
print(f"without data preprocessing algorithm has {virgin_data_mean_result/5} accuracy")
print(f"with data preprocessing algorithm has {filled_data_mean_result/5} accuracy")

without data preprocessing algorithm has 0.0 accuracy
with data preprocessing algorithm has 0.9030557677628076 accuracy
